In [1]:
import pandas as pd
import numpy as np
import lightgbm
from function import *

In [3]:
train_df = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/Data_/train_data.csv")
test_df = pd.read_csv("/Users/rish/Desktop/Kaggle/Novozymes_Enzyme_Stability_Prediction/Data/Data_/test_data.csv")

In [4]:
train_df, test_df = finalize_data(train_df, test_df)

In [6]:
from sklearn.model_selection import KFold

In [11]:
X_cols = train_df.drop("tm", axis = 1).columns

In [12]:
kf = KFold(n_splits=5)
for train_index, val_index in kf.split(train_df[X_cols]):
    X_train = train_df[X_cols].iloc[train_index]
    X_test = train_df[X_cols].iloc[val_index]
    y_train = train_df["tm"].iloc[train_index]
    y_test = train_df["tm"].iloc[val_index]


In [13]:
lgb_train = lightgbm.Dataset(X_train, y_train)
lgb_test = lightgbm.Dataset(X_test, y_test)

In [14]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

In [15]:
gbm = lightgbm.train(params,
                lgb_train,
                num_boost_round=20,
                valid_sets=lgb_test,
                callbacks=[lightgbm.early_stopping(stopping_rounds=5)])

[LightGBM] [Fatal] Unknown objective type name: classification


LightGBMError: Unknown objective type name: classification